In [1]:
# Instalar dependencia necesaria (solo necesario la primera vez)
!pip install cassandra-driver

In [2]:
import random
import uuid
import time
from datetime import datetime, timedelta
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement, SimpleStatement
from cassandra import ConsistencyLevel
from decimal import Decimal

In [3]:
# Configuración de conexión
# NOTA: Usamos 'cassandra' como host porque estamos dentro de la red de Docker.
# Si estuvieras fuera de Docker (tu PC), tendrías que exponer el puerto y usar 'localhost'.
CASSANDRA_HOST = ['cassandra'] 
KEYSPACE = 'ventas_db'
TABLE = 'ventas_crudas'
NUM_REGISTROS = 100000
BATCH_SIZE = 100

In [4]:
def setup_database():
    print("Conectando a Cassandra...")
    cluster = Cluster(CASSANDRA_HOST)
    session = cluster.connect()
    
    print(f"Creando Keyspace '{KEYSPACE}' si no existe...")
    session.execute(f"""
        CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
        WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': 1}};
    """)
    
    session.set_keyspace(KEYSPACE)
    
    print(f"Creando Tabla '{TABLE}' si no existe...")
    session.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE} (
            id_venta UUID,
            fecha_venta TIMESTAMP,
            id_producto TEXT,
            categoria TEXT,
            monto_total DECIMAL,
            id_cliente TEXT,
            PRIMARY KEY (fecha_venta)
        );
    """)
    
    return cluster, session

cluster, session = setup_database()
print("✅ Base de datos configurada correctamente.")

Conectando a Cassandra...
Creando Keyspace 'ventas_db' si no existe...
Creando Tabla 'ventas_crudas' si no existe...
✅ Base de datos configurada correctamente.


In [5]:
CATEGORIAS = ['Electronica', 'Ropa', 'Hogar', 'Alimentos', 'Deportes']

def generar_registro():
    fecha = datetime(2024, 1, 1) + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    return (
        uuid.uuid4(),                          # id_venta
        fecha,                                 # fecha_venta
        f"PROD-{random.randint(100, 999)}",    # id_producto
        random.choice(CATEGORIAS),             # categoria
        Decimal(str(round(random.uniform(5.0, 1500.0), 2))), # monto_total (Decimal)
        f"CLI-{random.randint(1, 500)}"        # id_cliente
    )

In [6]:
print(f"--- Iniciando Ingesta de {NUM_REGISTROS} registros ---")
start_time = time.time()

insert_query = session.prepare(f"""
    INSERT INTO {TABLE} (id_venta, fecha_venta, id_producto, categoria, monto_total, id_cliente)
    VALUES (?, ?, ?, ?, ?, ?)
""")

batch = BatchStatement(consistency_level=ConsistencyLevel.ONE)
count_in_batch = 0
total_inserted = 0

for _ in range(NUM_REGISTROS):
    datos = generar_registro()
    batch.add(insert_query, datos)
    count_in_batch += 1
    
    if count_in_batch >= BATCH_SIZE:
        session.execute(batch)
        batch = BatchStatement(consistency_level=ConsistencyLevel.ONE)
        count_in_batch = 0
        total_inserted += BATCH_SIZE
        
        if total_inserted % 10000 == 0:
            print(f"Insertados: {total_inserted}...")

# Insertar remanentes
if count_in_batch > 0:
    session.execute(batch)
    total_inserted += count_in_batch

end_time = time.time()
duration = end_time - start_time

print(f"✅ Ingesta Completada.")
print(f"Total registros: {total_inserted}")
print(f"Tiempo total: {duration:.2f} segundos")
print(f"Tasa: {total_inserted / duration:.2f} regs/seg")

--- Iniciando Ingesta de 100000 registros ---
Insertados: 10000...
Insertados: 20000...
Insertados: 30000...
Insertados: 40000...
Insertados: 50000...
Insertados: 60000...
Insertados: 70000...
Insertados: 80000...
Insertados: 90000...
Insertados: 100000...
✅ Ingesta Completada.
Total registros: 100000
Tiempo total: 9.06 segundos
Tasa: 11038.15 regs/seg


In [7]:
# Validación Final
row = session.execute(f"SELECT COUNT(*) FROM {TABLE}").one()
print(f"Registros en base de datos: {row[0]}")

cluster.shutdown()

Registros en base de datos: 280012
